In [42]:
import numpy as np
import math
from collections import namedtuple

from onms_decoder import OnmsDecoder
from awgn import Awgn
from bpsk import Bpsk
from awgn_llr_adapter import AwgnLlrAdapter


In [46]:
Settings = namedtuple('Settings', 'max_iter rejections_count snr_array')

In [59]:
def get_sigma(snr):
    return np.sqrt(10 ** (-snr/10)/2)

def simulate(settings, codeword, modulation, channel, decoder, decoder_adapter=None):
    
    for snr in settings.snr_array:
        
        sigma = get_sigma(snr)
        channel.sigma = sigma
        decoder_adapter.sigma = sigma
        
        errors_count = 0
        iterations_count = 0
        
        while (errors_count <= settings.rejections_count) and (iterations_count < settings.max_iter): 
            modulated = modulation.modulate(codeword);
            transmitted = channel.simulate(modulated);
            
            if decoder_adapter is not None:
                transmitted = decoder_adapter.adapt(transmitted)

            decoded = decoder.decode(transmitted)
            errors_count += not decoded[0]
            iterations_count += 1
        
        print(sigma)
        print(errors_count)
        print(iterations_count)
    return

In [61]:
H = np.array([[1, 1, 1, 1, 0, 0], [0, 0, 1, 1, 0, 1], [1, 0, 0, 1, 1, 0]])
codeword = np.array([0, 0, 1, 1, 1, 0])

decoder = OnmsDecoder(H);
channel = Awgn()
modulation = Bpsk()
adapter = AwgnLlrAdapter()

settings = Settings(1000000, 30, [-2, -1, -1.5, -0.5, 0, 0.5, 1, 1.5, 10])
simulate(settings, codeword, modulation, channel, decoder, adapter)

0.8901946956877225
31
154
0.7933868576533668
31
363
0.8403979844760322
31
177
0.7490054920699725
31
213
0.7071067811865476
31
307
0.6675518474746908
31
526
0.6302095820932436
31
656
0.5949562103147331
31
953
0.22360679774997896
0
1000001
